In [1]:
import numpy as np
import networkx as nx
from deap import base, creator, tools, algorithms
import random
import folium

# Define the coordinates of the hotspots (replace with actual hotspot coordinates)
hotspots = [
    (1.4330621251437414, 103.83778167687922),  # Yishun coordinates
    (1.4371190344748281, 103.83539436698997),
    (1.4252566436379683, 103.83590935114745),
    (1.4248276228794505, 103.84286163671058),
    (1.427916570233905, 103.83204697021661)
]

# Create a graph where nodes are hotspots and edges are Euclidean distances between them
G = nx.complete_graph(len(hotspots))
for i, coord1 in enumerate(hotspots):
    for j, coord2 in enumerate(hotspots):
        if i != j:
            distance = np.linalg.norm(np.array(coord1) - np.array(coord2))
            G[i][j]['weight'] = distance

# Check if the graph is created correctly
print("Graph Nodes:", G.nodes)
print("Graph Edges:", G.edges(data=True))

Graph Nodes: [0, 1, 2, 3, 4]
Graph Edges: [(0, 1, {'weight': 0.004707203185325589}), (0, 2, {'weight': 0.008026901343781687}), (0, 3, {'weight': 0.009675382133480553}), (0, 4, {'weight': 0.00770477746831806}), (1, 2, {'weight': 0.011873564125773988}), (1, 3, {'weight': 0.014381895427490602}), (1, 4, {'weight': 0.00979236504955768}), (2, 3, {'weight': 0.006965510272948608}), (2, 4, {'weight': 0.004689690389640978}), (3, 4, {'weight': 0.011247159958603328})]


In [2]:
# Create a Folium map centered around the first hotspot
map_center = hotspots[0]
mymap = folium.Map(location=map_center, zoom_start=13)

# Add crime hotspots to the map as markers
for i, coord in enumerate(hotspots):
    folium.Marker(location=coord, popup=f"Hotspot {i+1}").add_to(mymap)

# Add edges of the graph as lines to the map
for edge in G.edges(data=True):
    node1, node2, data = edge
    coord1, coord2 = hotspots[node1], hotspots[node2]
    folium.PolyLine(locations=[coord1, coord2], color='blue', weight=2.5, opacity=1).add_to(mymap)

# Save or display the map
mymap.save('graph_map.html')
mymap


In [3]:
import itertools

# Define the fitness function
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("PatrolPolice", list, fitness=creator.FitnessMin)

# Define the evaluation function
def eval_route(patrol_police):
    distance = 0
    for i in range(len(patrol_police) - 1):
        distance += G[patrol_police[i]][patrol_police[i + 1]]['weight']
    distance += G[patrol_police[-1]][patrol_police[0]]['weight']
    return distance,

# Genetic Algorithm setup
toolbox = base.Toolbox()
toolbox.register("indices", itertools.permutations, range(len(hotspots)))
toolbox.register("patrol_police", lambda: list(toolbox.indices()))
toolbox.register("mate", tools.cxPartialyMatched)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", eval_route)

# Create an initial population
population = [creator.PatrolPolice(p) for p in toolbox.patrol_police()]

# Evaluate the fitness of initial population
fitnesses = list(map(toolbox.evaluate, population))
for ind, fit in zip(population, fitnesses):
    ind.fitness.values = fit
    print("Initial individual:", ind)  # Debug print
    print("Fitness:", fit)  # Debug print

print("Number of hotspots:", len(hotspots))  # Debug print
print("Number of nodes in the graph:", len(G.nodes))  # Debug print


Initial individual: [0, 1, 2, 3, 4]
Fitness: (0.04249821501096957,)
Initial individual: [0, 1, 2, 4, 3]
Fitness: (0.04219299979282444,)
Initial individual: [0, 1, 3, 2, 4]
Fitness: (0.03844907674372384,)
Initial individual: [0, 1, 3, 4, 2]
Fitness: (0.04305285030484218,)
Initial individual: [0, 1, 4, 2, 3]
Fitness: (0.03583015103095341,)
Initial individual: [0, 1, 4, 3, 2]
Fitness: (0.040739139810216894,)
Initial individual: [0, 2, 1, 3, 4]
Fitness: (0.053234298323967665,)
Initial individual: [0, 2, 1, 4, 3]
Fitness: (0.05061537261119724,)
Initial individual: [0, 2, 3, 1, 4]
Fitness: (0.046871449562096636,)
Initial individual: [0, 2, 3, 4, 1]
Fitness: (0.040739139810216894,)
Initial individual: [0, 2, 4, 1, 3]
Fitness: (0.046566234343951504,)
Initial individual: [0, 2, 4, 3, 1]
Fitness: (0.04305285030484218,)
Initial individual: [0, 3, 1, 2, 4]
Fitness: (0.048325309544704174,)
Initial individual: [0, 3, 1, 4, 2]
Fitness: (0.0465662343439515,)
Initial individual: [0, 3, 2, 1, 4]
Fitness

In [10]:
algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2, ngen=50, verbose=False)

# Get the best route
best_patrol_police = tools.selBest(population, k=1)[0]  
best_route = [hotspots[i] for i in best_patrol_police]  


In [11]:
# Visualize the optimal route with annotations
map_center = [np.mean([coord[0] for coord in hotspots]), np.mean([coord[1] for coord in hotspots])]
mymap = folium.Map(location=map_center, zoom_start=13)

# Add crime hotspots to the map
for i, coord in enumerate(hotspots):
    folium.CircleMarker(location=coord, radius=5, color='red').add_to(mymap)
    folium.Marker(location=coord, icon=folium.Icon(color='green'), popup=f"Hotspot {i+1}").add_to(mymap)  # Add hotspot numbers

# Add the optimal patrol route to the map with annotations
for i in range(len(best_route)):
    coord = best_route[i]
    folium.Marker(location=coord, icon=folium.DivIcon(html=f"<div style='font-size: large;'>{i+1}</div>")).add_to(mymap)  # Add visit numbers
    if i < len(best_route) - 1:
        next_coord = best_route[i+1]
        folium.PolyLine(locations=[coord, next_coord], color='blue', weight=2.5, opacity=1).add_to(mymap)

# Connect the last hotspot to the starting point
folium.PolyLine(locations=[best_route[-1], best_route[0]], color='blue', weight=2.5, opacity=1).add_to(mymap)

# Save or display the map
mymap.save('optimal_patrol_route_with_annotations.html')
mymap
